# Logisticka regresija

### Predvidjanje da li cena bitcoina raste ili opada na kraju dana koriscenjem modela Logisticke regresije.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import classification_report

### Ucitavanje podataka

In [ ]:
data = pd.read_csv('data/bitstampUSD_1-min_data_2012-01-01_to_2021-03-31.csv')
data.head()

In [ ]:
data.shape

### Uklanjanje redova koji sadrze NaN vrednosti.

In [ ]:
data = data.dropna()
data.head()

### Uklanjanje nepotrebnih kolona.

In [ ]:
data_=data.drop(['Volume_(BTC)','Volume_(Currency)','Weighted_Price'], axis=1)
data_.head()

### Dodavanje kolone sa indeksom.

In [ ]:
#data_["row_id"]= range(1, len(data_) + 1)

In [ ]:
#data_.set_index('row_id', inplace=True)
#data_.head()

### Razlika vrednosti na otvaranju i vrednosti na zatvaranju.

In [ ]:
# da li se instanca (dan, svaki dan) zatvorila
# sa porastom vrednosti ili sa padom vrednosti
data_['pos_neg'] = data_['Open']-data_['Close']
data_.head()
# kreiranje nove binarne kolone cije vrednosti mogu biti:
# 0 = gubitak (pad cene)
# 1 = dobitak (rast cene)
# (to je ono sto u nastavku treba predvideti)
data_['Up/Down'] = np.where(data_['pos_neg']>0, 0, 1)

In [ ]:
data_.head()

In [ ]:
# TODO
plt.hist(data_['Up/Down'])

In [ ]:
data_ = data_.drop('Timestamp',axis=1)
data_.head()

In [ ]:
data_.shape

In [ ]:
data_.info()

### Stratifikovana podela skupa podataka na skup za treniranje i skup za testiranje u datoj razmeri.

In [ ]:
X = data_[['Open', 'High', 'Low','Close']]
y = data_['Up/Down']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=101, stratify = y)

In [ ]:
# vise je ne-nula elemenata, kao sto je i ocekivano
y.shape, np.count_nonzero(y, axis=0)

In [ ]:
non_zero_train = np.count_nonzero(y_train, axis=0) # ne-nula
zero_train = y_train.shape[0] - non_zero_train # nula
print('Jedinice: ', non_zero_train, 'Nule: ', zero_train)

In [ ]:
non_zero_test = np.count_nonzero(y_test, axis=0) # ne-nula
zero_test = y_test.shape[0] - non_zero_test # nula
print('Jedinice: ', non_zero_test, 'Nule: ', zero_test)

In [ ]:
plt.title("Vizuelna provera stratifikovanosti")
plt.xticks([0,1])
plt.xlabel(' 0 - pad cene, 1 - rast cene')
plt.hist([y_train, y_test], color=['orange', 'green'], label=['skup za treniranje', 'skup za testiranje'])
plt.legend(loc='best')
plt.show()

### Sve vrednosti su u istom intervalu. Skaliranje nije potrebno. # TODO

### Kreiranje i treniranje modela.

In [ ]:
logmodel = LogisticRegression()
logmodel.fit(X_train,y_train)

### Predvidjanje modela.

In [ ]:
predictions = logmodel.predict(X_test)
train_predictions = logmodel.predict(X_train)

### Tacnost na skupovima za treniranje i testiranje.

In [ ]:
train_score = metrics.accuracy_score(y_train, train_predictions)
test_score = metrics.accuracy_score(y_test, predictions)
print("Tacnost na skupu za treniranje: {train}\nTacnost na skupu za testiranje: {test}".format(train=train_score, test=test_score))

### Ostale mere na skupu za testiranje.

In [ ]:
print(classification_report(y_test,predictions))

### Matrica konfuzije.

In [ ]:
confusion_m = metrics.confusion_matrix(y_test, predictions)
confusion_m

In [ ]:
tp = confusion_m[0][0]
tp

In [ ]:
fp = confusion_m[0][1]
fp

In [ ]:
fn = confusion_m[1][0]
fn

In [ ]:
tn = confusion_m[1][1]
tn

### Analiza sigurnosti klasifikatora.

In [ ]:
first_ten_predictions = predictions[0:10]
first_ten_predictions

In [ ]:
# verovatnoce da instance pripadaju pozitivnoj odnosno negativnoj klasi
predictions_probabilities = logmodel.predict_proba(X_test)
predictions_probabilities[0:10]

In [ ]:
logmodel_confidence = []

for p in predictions_probabilities: 
    logmodel_confidence.append(np.abs(p[0]-p[1]))
    
logmodel_confidence = np.array(logmodel_confidence)

In [ ]:
# indeksi instanci uredjeni na osnovu sigurnosti klasifikatora u svoju odluku
# pocev od indeksa instanci oko cije klasifikacije je klasifikator najmanje siguran
logmodel_confidence.argsort()

In [ ]:
# uredjeni niz razlike verovatnoca pocev od onih koji se
# najmanje razlikuju ka onim koji se najvise razlikuju
logmodel_confidence.sort()
logmodel_confidence